<a href="https://colab.research.google.com/github/zhirnovv/image-segmentation/blob/main/dinitz/algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import inf
import glob

In [ ]:
class Edge():
  def __init__(self, capacity):
    self.capacity = capacity
    self.flow = 0

In [ ]:
class Graph():
  def __init__(self, vertice_amount):
    self.vertice_amount = vertice_amount
    self.adjacency_matrix = [[-inf] * vertice_amount for i in range(vertice_amount)];

  def add_edge(self, start, end, capacity):
    self.adjacency_matrix[start][end] = Edge(capacity)
    self.adjacency_matrix[end][start] = Edge(capacity)
    self.adjacency_matrix[end][start].flow = capacity

In [260]:
class MaxFlow():
  def __init__(self, graph, source, sink):
    self.graph = graph
    self.source = source
    self.sink = sink

    self.level = [-1] * graph.vertice_amount;
    self.pointers = [0] * graph.vertice_amount;

    self.level[source] = 0
    self.q = [source]

  def bfs(self):
    while self.q:
      current_vertice_index = self.q.pop()
      for adjacent_vertice_index in range(len(self.graph.adjacency_matrix[current_vertice_index])):
        current_edge = self.graph.adjacency_matrix[current_vertice_index][adjacent_vertice_index]
        
        if current_edge == -inf:
          continue

        if current_edge.capacity - current_edge.flow < 1:
          continue

        if self.level[adjacent_vertice_index] != -1:
          continue

        self.level[adjacent_vertice_index] = self.level[current_vertice_index] + 1
        self.q.insert(0, adjacent_vertice_index)

    return self.level[self.sink] != -1

  def dfs(self, start, flow_to_push):
    if flow_to_push == 0:
      return 0
    
    if start == self.sink:
      return flow_to_push
    
    pointer = self.pointers[start] # cid

    while pointer < len(self.graph.adjacency_matrix[start]):
      current_edge = self.graph.adjacency_matrix[start][pointer]
      reverse_edge = self.graph.adjacency_matrix[pointer][start]

      if current_edge == -inf:
        pointer += 1
        self.pointers[start] = pointer
        continue

      if self.level[start] +1 != self.level[pointer] or current_edge.capacity - current_edge.flow < 1:
        pointer += 1
        self.pointers[start] = pointer
        continue

      flow = self.dfs(pointer, min(flow_to_push, current_edge.capacity - current_edge.flow))

      if flow == 0:
        pointer += 1
        self.pointers[start] = pointer
        continue

      current_edge.flow += flow
      reverse_edge.flow -= flow

      return flow

    return 0
  
  def flow(self):
    max_flow = 0
    while True:
      self.level = [-1] * self.graph.vertice_amount;
      self.level[self.source] = 0
      self.q = [self.source]
      if (not self.bfs()):
        break
      self.pointers = [0] * self.graph.vertice_amount;
      pushed_flow = self.dfs(self.source, inf)
      while pushed_flow:
        max_flow += pushed_flow
        pushed_flow = self.dfs(self.source, inf)
    return max_flow


In [261]:
class TestSuite():
  def __init__(self):
    self.test_file_paths = glob.glob('/content/tests/*.txt')

  def throw_error(self, reason, explanation):
    raise Exception('Неверный формат тестового файла: ', reason, explanation)

  def run_test(self, file):
    print('---- НАЧАЛО ТЕСТА ----')
    print('Тестовый файл --', file, '\n')
    initial_graph_parameters = file.readline().strip().split()
    if len(initial_graph_parameters) != 2:
      self.throw_error('некорректные параметры графа', 'первая строка должна содержать число вершин и число ребер через пробел')
    if not all(param.isnumeric() for param in initial_graph_parameters):
      self.throw_error('некорректные параметры графа', 'параметры должны иметь целочисленный формат')
    
    vertice_amount, edge_amount = int(initial_graph_parameters[0]), int(initial_graph_parameters[1])
    graph = Graph(vertice_amount)

    print('Инициализирован граф, число вершин -', vertice_amount)
    print('Число ребер -', edge_amount)

    for line in range(edge_amount):
      print('Ребро', line + 1)
      edge_parameters = file.readline().strip().split()
      if len(edge_parameters) != 3:
        self.throw_error('некорректные параметры ребра', 'строка должна содержать начальную вершину, конечную вершину и пропускную способность через пробел')
      if not all(param.isnumeric() for param in edge_parameters):
        self.throw_error('некорректные параметры ребра', 'параметры должны иметь целочисленный формат')
      start, end, capacity = int(edge_parameters[0]) - 1, int(edge_parameters[1]) - 1, int(edge_parameters[2])
      print('Начальная вершина -', start, 'Конечная вершина -', end, 'Пропускная способность', capacity)
      graph.add_edge(start, end, capacity)
    
    dinitz = MaxFlow(graph, 0, vertice_amount - 1)

    print('\nМаксимальный поток -', dinitz.flow())
    print('---- КОНЕЦ ТЕСТА ---- \n')      


  def run_tests(self):
    for test_file_path in self.test_file_paths:
      test_file = open(test_file_path)
      self.run_test(test_file)





In [262]:
# Чтобы добавить тестовые файлы, необходимо:
# 1. Открыть слева вкладку "Файлы"
# 2. Создать директорию tests, если ее нет
# 3. Добавить в tests тестовые файлы в расширении .txt
# 4. Запустить все ячейки по порядку

TestSuite().run_tests()

---- НАЧАЛО ТЕСТА ----
Тестовый файл -- <_io.TextIOWrapper name='/content/tests/2.txt' mode='r' encoding='UTF-8'> 

Инициализирован граф, число вершин - 9
Число ребер - 15
Ребро 1
Начальная вершина - 0 Конечная вершина - 1 Пропускная способность 6
Ребро 2
Начальная вершина - 0 Конечная вершина - 2 Пропускная способность 1
Ребро 3
Начальная вершина - 0 Конечная вершина - 3 Пропускная способность 10
Ребро 4
Начальная вершина - 1 Конечная вершина - 2 Пропускная способность 2
Ребро 5
Начальная вершина - 3 Конечная вершина - 2 Пропускная способность 2
Ребро 6
Начальная вершина - 1 Конечная вершина - 4 Пропускная способность 4
Ребро 7
Начальная вершина - 1 Конечная вершина - 5 Пропускная способность 1
Ребро 8
Начальная вершина - 2 Конечная вершина - 5 Пропускная способность 20
Ребро 9
Начальная вершина - 3 Конечная вершина - 6 Пропускная способность 5
Ребро 10
Начальная вершина - 4 Конечная вершина - 5 Пропускная способность 2
Ребро 11
Начальная вершина - 5 Конечная вершина - 6 Пропускная сп